In case you are running this notebook in Colab! If you want to enable GPU acceleration, ensure you select a GPU runtime in the top-right dropdown menu 🚀

# Training ML potentials using GNNs

> **FYI**, you can open this documentation as a [Google Colab notebook](https://colab.research.google.com/github/omidshy/ml-notebooks/blob/master/notebooks/graph-pes-quickstart.ipynb) to follow along interactively

[graph-pes-train](https://jla-gardner.github.io/graph-pes/cli/graph-pes-train/root.html) provides a unified interface to train any [GraphPESModel](https://jla-gardner.github.io/graph-pes/models/root.html#graph_pes.GraphPESModel), including those packaged within [graph_pes.models](https://jla-gardner.github.io/graph-pes/models/root.html) and those defined by you, the user.

For more information on the ``graph-pes-train`` command, and the plethora of options available for specification in your ``config.yaml`` see the [CLI reference](https://jla-gardner.github.io/graph-pes/cli/graph-pes-train/root.html).

Below, we train a lightweight [MACE](https://jla-gardner.github.io/graph-pes/models/many-body/mace.html) model on the [C-GAP-17](https://jla-gardner.github.io/load-atoms/datasets/C-GAP-17.html) a dataset containing 4530 structures of amorphous carbon.

## Installation


In [ ]:
!pip install graph-pes

We now should have access to the ``graph-pes-train`` command. We can check this by running:

In [ ]:
!graph-pes-train -h

## Reference Data 

We use [load-atoms](https://jla-gardner.github.io/load-atoms/) to download and split the C-GAP-17 dataset into training, validation and test datasets:

In [ ]:
import ase.io
from load_atoms import load_dataset

structures = load_dataset("C-GAP-17")
train, val, test = structures.random_split([0.8, 0.1, 0.1])

ase.io.write("train-cgap17.xyz", train)
ase.io.write("val-cgap17.xyz", val)
ase.io.write("test-cgap17.xyz", test)

We can visualise the kinds of structures we're training on using [load_atoms.view](https://jla-gardner.github.io/load-atoms/api/viz.html):

In [ ]:
from load_atoms import view

view(train[0], show_bonds=True)

As you can see, each structure has an energy label:

In [ ]:
train[0].info["energy"]

... as well as a forces label (one for each atom in the structure):

In [ ]:
train[0].arrays["forces"].shape

These properties are stored in the files we have just created:

In [ ]:
!head train-cgap17.xyz

## Configuration

Great - now lets train a model. To do this, we have specified the following in our ``quickstart-cgap17.yaml`` file:

* the model architecture to instantiate and train, here [MACE](https://jla-gardner.github.io/graph-pes/models/many-body/mace.html). Note that we also include a [FixedOffset](https://jla-gardner.github.io/graph-pes/models/offsets.html#graph_pes.models.FixedOffset) component to account for the fact that the C-GAP-17 labels have an arbitrary energy offset.
* the data to train on, here the [C-GAP-17](https://jla-gardner.github.io/load-atoms/datasets/C-GAP-17.html) dataset we just downloaded
* the loss function to use, here a combination of a per-atom energy loss and a per-atom force loss
* and various other training hyperparameters (e.g. the learning rate, batch size, etc.)



We can download [this config file](https://raw.githubusercontent.com/omidshy/ml-notebooks/refs/heads/master/data/quickstart-cgap17.yaml) using wget:

In [ ]:
%%bash

if [ ! -f quickstart-cgap17.yaml ]; then
    wget https://tinyurl.com/quickstart-config -O quickstart-cgap17.yaml
fi

## Training

We use the downloaded config file to start the training.


In [ ]:
!graph-pes-train quickstart-cgap17.yaml general/run_id=train-mace

## Model Analysis

As part of the `graph-pes-train` run, the model was tested on the test set we specified in the config file (see the final section of the logs above).

To analyse the model in more detail, we first need to load it from disk. You can see from the command we used, and the training logs above, that the best model from the training run (i.e. the set of weights that gave the lowest validation loss) has been saved as `graph-pes-results/train-mace/model.pt`.

Let's load that best model, put it on the GPU for accelerated inference if available, and get it ready for evaluation:

In [ ]:
import torch
from graph_pes.models import load_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model = (
    load_model("graph-pes-results/train-mace/model.pt")  # load the model
    .to(device)  # move to GPU if available
    .eval()  # set to evaluation mode
)

The easiest way to use our model is to use the [GraphPESCalculator](https://jla-gardner.github.io/graph-pes/tools/ase.html#graph_pes.utils.calculator.GraphPESCalculator) to act directly on [ase.Atoms](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#module-ase.atoms) objects:

In [ ]:
calculator = best_model.ase_calculator()
calculator.calculate(test[0], properties=["energy", "forces", "stress"])
calculator.results

We can see from a single data point that our model has done a reasonable job of learning the PES:

In [ ]:
calculator.get_potential_energy(test[0]), test[0].info["energy"]

... and predicting the atomic forces:

In [ ]:
calculator.get_forces(test[0])[1], test[0].arrays["forces"][1]

``graph-pes`` provides a few utility functions for visualising model performance:

In [ ]:
import matplotlib.pyplot as plt
from graph_pes.utils.analysis import parity_plot

%config InlineBackend.figure_format = 'retina'

parity_plot(
    best_model,
    test,
    property="energy_per_atom",
    units="eV / atom",
    lw=0,
    s=12,
    color="crimson",
)
plt.xlim(-158.5, -155)
plt.ylim(-158.5, -155);

In [ ]:
parity_plot(
    best_model,
    test,
    property="forces",
    units="eV / Å",
    lw=0,
    s=2,
    alpha=0.5,
    color="crimson",
)

In [ ]:
from graph_pes.utils.analysis import dimer_curve

dimer_curve(best_model, system="CC", units="eV", rmin=0.85, rmax=4.0);